In [22]:
import pandas as pd
from collections import Counter


def filter_records(record, param):
    return record[0][-1] != param

# Mengakses file Excel dengan path baru
file_path = 'Book5.xlsx'

# Membaca file Excel
excel_data = pd.read_excel(file_path, sheet_name=None)

# Menjadikan sheet QC sebagai database
qc_data = excel_data['QC']

# Menulis ulang pada sheet OUTPUT
output_data = qc_data.copy()  # Meng-copy data dari sheet QC ke OUTPUT

# Menulis ulang header pada sheet OUTPUT
new_headers = ['UWI', 'Date', 'Perf Status', 'Perf Top (ftMD)', 'Perf Base (ftMD)', 'PERFO SAND', 'Bottom',
               'Validasi', 'Selisih Bottom', 'T_TET1', 'T_TET2', 'T_TET3', 'T_TET4', 'T_TET5', 'T_TET6',
               'T_TET7', 'T_TET8', 'T_TET9', 'T_TET10', 'T_BKA', 'T_BKB', 'T_BKC1', 'T_BKC2', 'T_BKC3', 'T_BST']

# Memeriksa panjang header baru dan kolom pada data
if len(new_headers) == len(output_data.columns):
    output_data.columns = new_headers
else:
    print("Panjang header baru tidak sesuai dengan panjang kolom pada data. Tidak dapat melanjutkan.")
    exit()

# Menyimpan perubahan pada sheet OUTPUT
excel_data['OUTPUT'] = output_data

# Menampilkan data pada sheet TANDA
tanda_data = excel_data['TANDA']

uwi_and_time_output = []
for i in output_data.index:
    if [output_data['UWI'][i], output_data['Date'][i]] in uwi_and_time_output:
        continue
    else:
        uwi_and_time_output.append(
            [output_data['UWI'][i], output_data['Date'][i]])
results = []
hasil = []
sand_final = []
# Iterasi untuk mengisi nilai "P" pada kolom PERFO SAND di sheet TANDA
for index, row in tanda_data.iterrows():
    uwi = row['UWI']
    date = row['Date']

    # Temukan nilai PERFO SAND yang sesuai dengan UWI dan Date dari sheet OUTPUT_UNIQUE
    rs = []
    for i in output_data.index:
        if [output_data['UWI'][i] , output_data['Date'][i]] == [uwi, date]:
            if output_data['Perf Status'][i] == 'perforation':
                rs.append(([output_data['Perf Base (ftMD)'][i] - output_data['Perf Top (ftMD)'][i],
                    output_data['PERFO SAND'][i], output_data['Perf Status'][i], date, uwi]))
            elif output_data['Perf Status'][i] == 'squeeze':
                rs.append(([output_data['Perf Top (ftMD)'][i] - output_data['Perf Base (ftMD)'][i],
                            output_data['PERFO SAND'][i], output_data['Perf Status'][i], date, uwi]))
    results.append(rs)
    
    sand = []
    for i in rs:
        if i[1] in sand:
            continue
        else:
            sand.append(i[1])
    sand_final.append(sand)

for i in range(len(sand_final)):
    selisih_sand = []
    for j in sand_final[i]:
            selisih = []
            date_sand = []
            uwi_sand = []
            for k in results[i]:
                if j == k[1]:
                    selisih.append(k[0])
            selisih = sum(selisih)
            date_sand = results[i][0][3]
            uwi_sand = results[i][0][4]
            selisih_sand.append([j, selisih, date_sand, uwi_sand])
    hasil.append(selisih_sand)
    
hasil_akhir = []
count = 0
for i in range(len(hasil)):
    count += 1
    for j in hasil[i]:
        found = False
        if len(hasil_akhir) != 0:
            if j[3] != hasil_akhir[0][0][3]:
                keyword = hasil_akhir[0][0][3]
                filtered_data = [
                    inner_list for inner_list in hasil_akhir if inner_list[0][-1] != keyword]
                hasil_akhir = filtered_data
            else:
                for k, sublist in enumerate(hasil_akhir):
                    if j[0] == sublist[0][0] and j[3] == sublist[0][3]:
                        hasil_akhir[k][0][1] += j[1]
                        found = True
                        for l in hasil_akhir:
                            if l[0][1] == 0:
                                l[1] = "squeeze"
                                l[0][3] = j[3]
                                l[0][2] = j[2]
                            else:
                                l[1] = "perforation"
                                l[0][3] = j[3]
                                l[0][2] = j[2]
                        break
        if not found and j[1] != 0:
            hasil_akhir.append([j,"perforation"])
        elif not found and j[1] == 0:
            hasil_akhir.append([j,"squeeze"])
    for i in hasil_akhir:
        print(i)
    print('break ' + str(count))

# # Menyimpan perubahan pada sheet TANDA
# excel_data['TANDA'] = tanda_data

# # Menyimpan ke file Excel
# with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
#     for sheet, data in excel_data.items():
#         data.to_excel(writer, sheet_name=sheet, index=False)

[['T_BKA', 9, Timestamp('1971-07-12 00:00:00'), 'PETP00001'], 'perforation']
[['T_BKB', 8, Timestamp('1971-07-12 00:00:00'), 'PETP00001'], 'perforation']
[['T_BKC1', 46, Timestamp('1971-07-12 00:00:00'), 'PETP00001'], 'perforation']
[['T_BKC2', 43, Timestamp('1971-07-12 00:00:00'), 'PETP00001'], 'perforation']
[['T_BST', 16, Timestamp('1971-07-12 00:00:00'), 'PETP00001'], 'perforation']
break 1
[['T_BKA', 9, Timestamp('1973-03-31 00:00:00'), 'PETP00001'], 'perforation']
[['T_BKB', 8, Timestamp('1973-03-31 00:00:00'), 'PETP00001'], 'perforation']
[['T_BKC1', 46, Timestamp('1973-03-31 00:00:00'), 'PETP00001'], 'perforation']
[['T_BKC2', 43, Timestamp('1973-03-31 00:00:00'), 'PETP00001'], 'perforation']
[['T_BST', 0, Timestamp('1973-03-31 00:00:00'), 'PETP00001'], 'squeeze']
break 2
[['T_BKA', 9, Timestamp('1973-05-07 00:00:00'), 'PETP00001'], 'perforation']
[['T_BKB', 8, Timestamp('1973-05-07 00:00:00'), 'PETP00001'], 'perforation']
[['T_BKC1', 46, Timestamp('1973-05-07 00:00:00'), 'PETP